<a href="https://colab.research.google.com/github/ToshikiShimizu/PyTorch/blob/master/intro_dev/chapter4/4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch
!pip install torchvision

    100% |████████████████████████████████| 519.5MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5877a000 @  0x7ff682fe02a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 2.0MB 8.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [15]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar xf 102flowers.tgz
#!mkdir oxford-102
#!mkdir oxford-102/jpg
!mv jpg/*.jpg oxford-102/jpg

--2018-11-14 13:44:02--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2018-11-14 13:44:03 ERROR 403: Forbidden.

tar: 102flowers.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
mv: cannot stat 'jpg/*.jpg': No such file or directory


In [0]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
from torch import nn

In [8]:
img_data = ImageFolder("oxford-102/",transform=transforms.Compose([transforms.Resize(80),transforms.CenterCrop(64),transforms.ToTensor()]))
batch_size = 64
img_loader = DataLoader(img_data, batch_size = batch_size, shuffle=True)

RuntimeError: ignored

In [0]:
from torch import nn

In [0]:
nz = 100
ngf = 32

class GNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.main = nn.Sequential(
    nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
    nn.BatchNorm2d(ngf*8),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf*2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf*2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf*2),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
    nn.Tanh()
    )
  def forward(self, x):
    out = self.main(x)
    return out

In [2]:
ndf = 32
class DNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.main = nn.Sequential(
    nn.Conv2d(3, ndf, 4, 2, 1 ,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf*2),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf*4),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf*8),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
    )
  def forward(self, x):
    out = self.main(x)
    return out.squeeze()
        
        

NameError: ignored

In [0]:
!ls oxford-102

In [0]:
d = DNet().to("cuda:0")
g = GNet().to("cuda:0")
opt_d = optim.Adam(d.parameters(),
                  lr = 0.0002, betas=(0.5, 0.9999))
opt_g = optim.Adam(g.parameters(),lr = 0.0002, betas=(0.5, 0.999))

ones = torch,ines(batch_size).to("cuda:0")
zeros = torch.zeros(batch_size).to("cuda:0")
loss_f = nn.BCEWithLogitsLoss()
fixed_z = torch.randn(batch_size, nz, 1, 1).to("cuda:0")